In [138]:
import os
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO

In [139]:
SCORE_DIR = "data/scores"

In [140]:
box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [151]:
def parse_html(box_score):
    with open(box_score, encoding="utf-8") as f:
        html = f.read()

    soup = BeautifulSoup(html)
    # remove the html elements tr.over_header and tr.thead
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [152]:
def read_season_info(soup):
    # find all a tags
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [153]:
def read_line_score(soup):
    # getting the teams that won and lost and the total points of each team
    html_str = str(soup)
    line_score = pd.read_html(StringIO(html_str), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    
    return line_score

In [154]:
def read_stats(soup, team, stat):
    # read the advanced stats for the game
    html_str = str(soup)
    df = pd.read_html(StringIO(html_str), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [ ]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1, :], advanced.iloc[-1, :]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]

        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0, 1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 11520
200 / 11520
300 / 11520
400 / 11520
500 / 11520
600 / 11520
700 / 11520
800 / 11520
900 / 11520
1000 / 11520
1100 / 11520
1200 / 11520
1300 / 11520
1400 / 11520
1500 / 11520
1600 / 11520
1700 / 11520
1800 / 11520
1900 / 11520
2000 / 11520
2100 / 11520
2200 / 11520
2300 / 11520
2400 / 11520
2500 / 11520
2600 / 11520
2700 / 11520
2800 / 11520
2900 / 11520
3000 / 11520
3100 / 11520
3200 / 11520


In [93]:
games_df = pd.concat(games, ignore_index=True)

In [94]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,265.0,265.0,39.0,98.0,0.398,12.0,38.0,0.316,20.0,31.0,...,50.0,42.9,156.0,108.0,ATL,108,1,2016,2016-04-01,True
1,265.0,265.0,38.0,95.0,0.400,9.0,30.0,0.300,23.0,26.0,...,38.1,30.1,147.0,104.0,CLE,110,0,2016,2016-04-01,False
2,240.0,240.0,33.0,90.0,0.367,10.0,39.0,0.256,15.0,21.0,...,27.6,30.1,157.0,102.0,CHO,100,1,2016,2016-04-01,False
3,240.0,240.0,36.0,84.0,0.429,7.0,27.0,0.259,21.0,27.0,...,30.8,30.1,136.0,109.0,PHI,91,0,2016,2016-04-01,True
4,240.0,240.0,39.0,85.0,0.459,8.0,24.0,0.333,12.0,14.0,...,50.0,33.0,169.0,115.0,DET,89,1,2016,2016-04-01,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,240.0,240.0,41.0,100.0,0.410,5.0,22.0,0.227,14.0,17.0,...,39.3,23.2,135.0,101.0,SAC,105,0,2016,2016-04-11,False
166,240.0,240.0,36.0,81.0,0.444,11.0,32.0,0.344,18.0,22.0,...,100.0,32.7,178.0,122.0,UTA,92,1,2016,2016-04-11,True
167,240.0,240.0,35.0,77.0,0.455,9.0,33.0,0.273,13.0,18.0,...,21.4,25.3,156.0,111.0,DAL,101,0,2016,2016-04-11,False
168,240.0,240.0,36.0,80.0,0.450,10.0,27.0,0.370,8.0,9.0,...,20.2,34.0,124.0,108.0,IND,102,1,2016,2016-04-12,False
